In [45]:
import pandas as pd 
import numpy as np
from weather import data
import datetime
import plotly.graph_objects as go
import plotly.express as px
import matplotlib.pyplot as plt
import requests
from dotenv import load_dotenv
import os


In [46]:
load_dotenv()
OMW_ENDPOINT = os.getenv('OMW_ENDPOINT')
API_KEY = os.getenv('API_KEY')

weather_params = {
    'lat':  54.51,
    'lon': 18.53,
    'cnt': 8,
    'appid': API_KEY,
}
data = requests.get(OMW_ENDPOINT, params=weather_params)
data = data.json()
print(data)

{'cod': '200', 'message': 0, 'cnt': 8, 'list': [{'dt': 1726066800, 'main': {'temp': 292.36, 'feels_like': 291.77, 'temp_min': 290.76, 'temp_max': 292.36, 'pressure': 1003, 'sea_level': 1003, 'grnd_level': 995, 'humidity': 55, 'temp_kf': 1.6}, 'weather': [{'id': 801, 'main': 'Clouds', 'description': 'few clouds', 'icon': '02d'}], 'clouds': {'all': 20}, 'wind': {'speed': 7.11, 'deg': 241, 'gust': 12.19}, 'visibility': 10000, 'pop': 0, 'sys': {'pod': 'd'}, 'dt_txt': '2024-09-11 15:00:00'}, {'dt': 1726077600, 'main': {'temp': 290.99, 'feels_like': 290.47, 'temp_min': 288.26, 'temp_max': 290.99, 'pressure': 1004, 'sea_level': 1004, 'grnd_level': 997, 'humidity': 63, 'temp_kf': 2.73}, 'weather': [{'id': 802, 'main': 'Clouds', 'description': 'scattered clouds', 'icon': '03n'}], 'clouds': {'all': 47}, 'wind': {'speed': 5.01, 'deg': 242, 'gust': 9.1}, 'visibility': 10000, 'pop': 0, 'sys': {'pod': 'n'}, 'dt_txt': '2024-09-11 18:00:00'}, {'dt': 1726088400, 'main': {'temp': 288.27, 'feels_like': 2

In [60]:
#Getting data from API
date = data['list'][0]['dt']
dates = [day['dt'] for day in data['list']]
temp_max = [day['main']['temp_max'] for day in data['list']]
temp_max = [temp - 273.15 for temp in temp_max]
temp_min = [day['main']['temp_min'] for day in data['list']]
temp_min = [temp - 273.15 for temp in temp_min]
pressure = [day['main']['pressure'] for day in data['list']]
humidity = [day['main']['humidity'] for day in data['list']]
wind = [day['wind']['speed'] for day in data ['list']]

In [59]:
#Convert dates
def convert_utc_list(dates):
    return [datetime.datetime.utcfromtimestamp(ts).strftime('%Y-%m-%d %H') for ts in dates]
converted_dates = convert_utc_list(dates)
for date in converted_dates:
    print(date)   
converted_dates = convert_utc_list(dates)
                 

2024-09-11 15
2024-09-11 18
2024-09-11 21
2024-09-12 00
2024-09-12 03
2024-09-12 06
2024-09-12 09
2024-09-12 12


In [54]:
#Make a dataframe of whole data
dict = {'Date': converted_dates, 'Temp_Max': temp_max, 'Temp_Min':temp_min, 'Humidity': humidity, 'Pressure': pressure, 'Wind_Speed': wind}
weather_forecast = pd.DataFrame(dict)
weather_forecast.set_index('Date', inplace=True)

In [55]:
weather_forecast.head()

,Temp_Max,Temp_Min,Humidity,Pressure,Wind_Speed
Date,,,,,
2024-09-11 15,19.21,17.61,55,1003,7.11
2024-09-11 18,17.84,15.11,63,1004,5.01
2024-09-11 21,15.12,13.07,71,1006,3.68
2024-09-12 00,12.73,12.73,76,1008,2.98
2024-09-12 03,11.36,11.36,83,1009,3.57


In [56]:
#Max temperature representation
fig = go.Figure()
fig.add_trace(go.Scatter(x=weather_forecast.index, y=weather_forecast['Temp_Max'], mode='lines+markers', name='Max Temperature (°C)'))
fig.update_traces(line_color='gold', fill='tozeroy')
fig.update_layout(template='simple_white')
fig.update_layout(width=1000, height=800)
fig.write_image('temperature.png')
fig.show()

In [57]:
#Humidity representation
fig_hum = px.line(weather_forecast, x=weather_forecast.index, y='Humidity', template='simple_white')
fig_hum.update_yaxes(range=[0, weather_forecast['Humidity'].max()])
fig_hum.update_traces(line_color='yellowgreen', fill='tozeroy')
fig_hum.write_image('humidity.png')
fig_hum.show()

In [58]:
#Wind speed representation
fin_wind = px.line(weather_forecast, x=weather_forecast.index, y='Wind_Speed', template='simple_white')
fin_wind.update_traces(line_color='turquoise', fill='tozeroy')
fin_wind.write_image('windspeed.png')
fin_wind.show()